In [ ]:
%matplotlib inline
from matplotlib import pylab as plt
from skimage import io, measure, exposure
from skimage.color import rgb2gray
import cv2
import numpy as np
import math

In [ ]:
img = cv2.imread("43.jpg")
image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(image_gray[0][0])

In [ ]:
cv2.imshow("image", image_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def hasWhiteBG(l)->bool:
    if l < 80:
        return False
    return True

def distance(p, x, y):
    return np.sqrt((p[0] - x)**2 + (p[1] - y)**2)

def getSize(x, y):
    return (np.max(x) - np.min(x), np.max(y) - np.min(y))

def getBounds(x, y):
    return (np.min(x), np.max(x), np.min(y), np.max(y))

In [ ]:
def getCenter(center, approx, xm, ym, xs, ys, im_s_x, im_s_y):
        if len(center) == 0:
            return {'a':approx, 'x':xmean, 'y':ymean, 'xs':xs, 'ys':ys}
        elif 2*im_s_x/3 > xm and im_s_x/3 < xm and 2*im_s_y/3 > ym and im_s_y/3 < ym and (xs > center['xs'] or ys > center['ys']):
            return {'a':approx, 'x':xmean, 'y':ymean, 'xs':xs, 'ys':ys}
        else:
            return center

In [ ]:
img = cv2.imread('43.jpg', cv2.IMREAD_GRAYSCALE)
image_gray = cv2.GaussianBlur(img, (15, 15), 1)

_, threshold = cv2.threshold(image_gray, 160, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

print(len(image[0]), len(image))
#cv2.circle(image, (169, 154), radius=4, color=(255,0,0), thickness=-1)

cv2.imshow('kontury', threshold)
cv2.waitKey(0)
cv2.destroyAllWindows()

acc = 25
bcr = 0.9
#cv2.circle(image, (587, 171), radius=4, color=(255,255,255), thickness=-1)
my_cnt = []
circles = []
center = {}

print(len(contours))
for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.001 * cv2.arcLength(contour, True), True)
    #print(approx)
    
    #cv2.drawContours(image, [approx], 0, (0,0,255), 2)
    
    x = approx.ravel()[::2]
    y = approx.ravel()[1::2]
    xs, ys = getSize(x, y)
    if (xs > bcr*len(image[0]) and ys > bcr*len(image)) or xs < len(image[0])/100 or ys < len(image)/100:
        continue
    
    xmean = np.mean(x)
    ymean = np.mean(y)
    #print(xs, ys, xmean, ymean)
        
    if xs < len(image[0])/2 and ys < len(image)/2:
        #print("new center ",xmean, ymean, "size ", xs, ys )
        
        center = getCenter(center, approx, xmean, ymean, xs, ys, len(image[0]), len(image))
            
    
    r1 = []
    for point in approx:
        r1.append(distance(point[0], xmean, ymean))
    print("minmax: ", max(r1)/min(r1))
    r1 = r1[8:-8]
    if max(r1) / min(r1) > 1.75:
        print("continue")
        continue
    r1 = np.mean(r1)
    print(r1)
    
    
    
    for cnt in my_cnt:
        print( xmean, ymean, cnt['x'], cnt['y'], hasWhiteBG(threshold[int(ymean)][int(xmean)]))
        cv2.circle(image, (int(xmean), int(ymean)), radius=4, color=(0,0,0), thickness=-1)
        #cv2.circle(image, (int(cnt['x']), int(cnt['y'])), radius=4, color=(0,0,0), thickness=-1)
        if (abs(cnt['x'] - xmean) < acc) and (abs(cnt['y'] - ymean) < acc) and hasWhiteBG(threshold[int(ymean)][int(xmean)]):
            """print("KOLKO!!!")
            print(xmean, ymean, cnt['x'], cnt['y'])
            print("obraz xy: ", (image[int(ymean)][int(xmean)]))
            print("obraz gray: ", (image_gray[int(ymean)][int(xmean)]))
            print("has WBG: ", hasWhiteBG(image_gray[int(ymean)][int(xmean)]))"""
            
            r2 = cnt['r']
            #print(r1, r2)
            
            if r1 > 3 and r2 > 3:
                #print("len app: ", len(approx))
                circles.append({'a':approx, 'x':xmean, 'y':ymean, 'r':r1} if r1 > r2 else cnt)
    if r1 > 3: 
        print("XXX")
        my_cnt.append({'a':approx, 'x':xmean, 'y':ymean, 'r':r1})
        #cv2.drawContours(image, [approx], 0, (0,0,255), 2)
    

rmean = 0
for circle in circles:
    #print(circle['r'])
    #print(len(circle['a']))
    rmean += circle['r']

rmean /= len(circles)

#print("rmean: ", rmean)
circles = [circle for circle in circles if (circle['r'] - acc < rmean and len(circle['a']) > 10)]

cv2.drawContours(image, [center['a']], 0, (0,255,0), 2)
#cv2.circle(image, (int(center['x']), int(center['y'])), radius=8, color=(0,0,0), thickness=-1)
   
#print(center['xs'], center['ys'])
    
for circle in circles:
    cv2.drawContours(image, [circle['a']], 0, (240,33,185), 2)
    


cv2.imshow('kontury', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def returnPosition(xc=0, yc=0, x=0, y=0, a=10):
    xr=-1
    yr=-1
    if x+a < xc:
        xr = 0
        
        if y+a < yc:
            yr = 0
        elif y-a > yc:
            yr = 2
        else:
            yr = 1
        
        return xr,yr
    elif x-a > xc:
        xr = 2
        
        if y+a < yc:
            yr = 0
        elif y-a > yc:
            yr = 2
        else:
            yr = 1
        
        return xr,yr
    
    else:
        xr = 1
        
        if y+a < yc:
            yr = 0
        elif y-a > yc:
            yr = 2
        else:
            yr = 1
        
        return xr,yr

In [ ]:
cent = {'x':100, 'y':100}
circ = [{'x':23, 'y':27}, {'x':104, 'y':26}, {'x':170, 'y':96}, {'x':107, 'y':179}]
cross = [{'x':26, 'y':174}, {'x':102, 'y':99}, {'x':171, 'y':26}]

board = [['-' for _ in range(3)] for _ in range(3)]

for c in circ:
    x, y = returnPosition(100, 100, c['x'], c['y'])
    board[y][x] = 'O'
    print(x, y)
        
for c in cross:
    x, y = returnPosition(100, 100, c['x'], c['y'])
    board[y][x] = 'X'
    print(x, y)
    
for l in board:
    print(l)


In [ ]:
def findCircles(image, original, offset_x, offset_y):
    print("FFFFFFFFF")
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _, threshold = cv2.threshold(image_gray, 120, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    acc = 15
    #cv2.circle(image, (587, 171), radius=4, color=(255,255,255), thickness=-1)
    my_cnt = []
    circles = []
    center = {}
    for contour in contours[0:]:
        approx = cv2.approxPolyDP(contour, 0.001 * cv2.arcLength(contour, True), True)
        #print(approx)
        x = approx.ravel()[::2]
        y = approx.ravel()[1::2]
        xs, ys = getSize(x, y)
        
        xmean = np.mean(x) #TODO usunąć
        ymean = np.mean(y)
        
        if xs < len(image[0])/100 or ys < len(image)/100:
            #cv2.circle(image, (int(xmean), int(ymean)), radius=4, color=(255,0,0), thickness=-1)
            continue

        #xmean = np.mean(x)
        #ymean = np.mean(y)
        #print(xs, ys, xmean, ymean)

        if xs < 2*len(image[0])/3 and ys < 2*len(image)/3:
            #print("center")
            center = getCenter(center, approx, xmean, ymean, xs, ys, len(image[0]), len(image))


        r1 = []
        for point in approx:
            r1.append(distance(point[0], xmean, ymean))
        #print("minmax: ", max(r1)/min(r1))
        if max(r1) / min(r1) > 1.75:
            continue
        r1 = np.mean(r1)


        #cv2.circle(image, (int(xmean), int(ymean)), radius=4, color=(0,0,0), thickness=-1)

        for cnt in my_cnt:
            #print("TŁO   ", threshold[int(ymean)][int(xmean)]) 
            if (abs(cnt['x'] - xmean) < acc) and (abs(cnt['y'] - ymean) < acc) and hasWhiteBG(threshold[int(ymean)][int(xmean)]):
                """print("KOLKO!!!")
                print(xmean, ymean, cnt['x'], cnt['y'])
                print("obraz xy: ", (image[int(ymean)][int(xmean)]))
                print("obraz gray: ", (image_gray[int(ymean)][int(xmean)]))
                print("has WBG: ", hasWhiteBG(image_gray[int(ymean)][int(xmean)]))"""

                r2 = cnt['r']
                #print(r1, r2)

                if r1 > 5 and r2 > 5:
                    #print("len app: ", len(approx))
                    circles.append({'a':approx, 'x':xmean, 'y':ymean, 'r':r1} if r1 > r2 else cnt)
        if r1 > 5: 
            my_cnt.append({'a':approx, 'x':xmean, 'y':ymean, 'r':r1})
            #cv2.drawContours(image, [approx], 0, (0,0,255), 2)




    rmean = 0
    for circle in circles:
        #print(circle['r'])
        #print(len(circle['a']))
        rmean += circle['r']

    rmean /= len(circles)

    #print("rmean: ", rmean)
    circles = [circle for circle in circles if (circle['r'] - acc < rmean and len(circle['a']) > 10)]

    cv2.drawContours(image, [center['a']], 0, (0,255,0), 2)
    cv2.drawContours(original, [center['a']], 0, (0,255,0), 2, offset=(offset_x, offset_y))
    #cv2.circle(image, (int(center['x']), int(center['y'])), radius=8, color=(0,0,0), thickness=-1)

    #print(center['xs'], center['ys'])

    for circle in circles:
        cv2.drawContours(image, [circle['a']], 0, (240,33,185), 2)
        cv2.drawContours(original, [circle['a']], 0, (240,33,185), 2, offset=(offset_x, offset_y))

In [ ]:
def isBoard(region):
    edges = cv2.GaussianBlur(region['thresh'], (15, 15), 2)
    edges = cv2.Canny(edges, 50, 50, apertureSize=3)
    
    cv2.imshow('kontury', edges)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    ml = (len(region['thresh']) + len(region['thresh'][0]))/(2 * 3)
    lines = cv2.HoughLinesP(edges, rho=1, theta=1 * np.pi / 180, threshold=50, minLineLength=ml, maxLineGap=25)
    
    angles = []
    
    for [[x1,y1,x2,y2]] in lines:
        angle = math.atan2(x2 - x1, y2 - y1)
        angle = angle * 180 / 3.14
        print(angle)
        angles.append(angle % 180)
        cv2.line(region['img'], (x1, y1), (x2, y2), (0, 0, 255), 2)
        
    cv2.imshow('kontury', region['img'])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if len(angles) > 8:
        return False
    
    for i in range(len(angles)):
        for j in range(i+1, len(angles)):
            if abs(angles[i]-angles[j]) < 5:
                return True
            
    return False

In [ ]:
#PLANSZE

img = cv2.imread('43.jpg', cv2.IMREAD_GRAYSCALE)
image_gray = cv2.GaussianBlur(img, (15, 15), 1)
print("image size: x: ",len(img[0]), "y: ", len(img))
_, threshold = cv2.threshold(image_gray, 120, 255, cv2.THRESH_BINARY)
contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

treshold = cv2.Canny(threshold, 50, 50, apertureSize=3)

cv2.imshow('kontury', threshold)
cv2.waitKey(0)
cv2.destroyAllWindows()

bcr = 0.9# biggest_contour_ratio
bsr = 1.5 # board size ratio
bsa = 1. # board size accuracy
ard = 40 # accuracy ratio divider

my_cnt = []
regions = []
reg_pics = []

xss = 0 # x size sum
yss = 0 # y size sum

for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.001 * cv2.arcLength(contour, True), True)
    #cv2.drawContours(image, [approx], 0, (0,0,255), 2)
    
    x = approx.ravel()[::2]
    y = approx.ravel()[1::2]
    
    xmin, xmax, ymin, ymax = getBounds(x, y)
    xs = xmax-xmin
    ys = ymax-ymin
    #print(xs, ys)
    if (xs > bcr*len(image[0]) and ys > bcr*len(image)) or xs < len(image[0])/100 or ys < len(image)/100:
        continue
    
    xss += xs
    yss += ys
    
    xm = np.mean(x) # x mean
    ym = np.mean(y) # y mean
    
    
    my_cnt.append({'a':approx, 'xmin':xmin, 'xmax':xmax, 'ymin':ymin,'ymax':ymax, 'xs':xs, 'ys':ys, 'xm':xm, 'ym':ym})
    


xsm = xss/len(my_cnt) # x size mean
ysm = yss/len(my_cnt) # y size mean
print('xsm: ', xsm, 'ysm: ', ysm)

for cnt in my_cnt:
    #cv2.circle(image, (int(cnt['xm']), int(cnt['ym'])), radius=8, color=(0,0,0), thickness=-1)
    if cnt['xs'] > bsr*xsm and cnt['ys'] > bsr*ysm:
        regions.append(dict(cnt))

for reg in regions:
    flags = [0, 0, 0, 0]
    xacc = reg['xs'] / ard
    yacc = reg['ys'] / ard
    for cnt in my_cnt:
        if cnt['ym']-yacc < reg['ymax'] and cnt['ym']+yacc > reg['ymin']:
        
            if cnt['xmin'] < reg['xmin'] and cnt['xm']+xacc > reg['xmin']:
                reg['xmin'] = cnt['xmin'] - 2*xacc
                flags[0] = 1

            elif cnt['xmax'] > reg['xmax'] and cnt['xm']-xacc < reg['xmax']:
                reg['xmax'] = cnt['xmax'] + 2*xacc
                flags[1] = 1
        
        if cnt['xm']-xacc < reg['xmax'] and cnt['xm']+xacc > reg['xmin']:
            
            if cnt['ymin'] < reg['ymin'] and cnt['ym']+yacc > reg['ymin']:
                reg['ymin'] = cnt['ymin'] - 2*yacc
                flags[2] = 1

            elif cnt['ymax'] > reg['ymax'] and cnt['ym']-yacc < reg['ymax']:
                reg['ymax'] = cnt['ymax'] + 2*yacc
                flags[3] = 1
    
    if flags[0] == 0:
        reg['xmin'] -= xacc
    if flags[1] == 0:
        reg['xmax'] += xacc
    if flags[2] == 0:
        reg['ymin'] -= yacc
    if flags[3] == 0:
        reg['ymax'] += yacc
    
    reg['xmin'] = int(reg['xmin']) if reg['xmin'] > 0 else 0
    reg['xmax'] = int(reg['xmax']) if reg['xmax'] < len(image[0]) else len(image[0])
    reg['ymin'] = int(reg['ymin']) if reg['ymin'] > 0 else 0
    reg['ymax'] = int(reg['ymax']) if reg['ymax'] < len(image) else len(image)
      

for r in regions:
    pt1 = (r['xmin'], r['ymin'])
    pt2 = (r['xmax'], r['ymax'])
    
    reg_pics.append({'img':image[r['ymin']:r['ymax'], r['xmin']:r['xmax']].copy(), 'x':r['xmin'], 'y':r['ymin'], 
                     'thresh':threshold[r['ymin']:r['ymax'], r['xmin']:r['xmax']].copy(), 'pt1': pt1, 'pt2': pt2})
    

    cv2.rectangle(image, pt1=pt1, pt2=pt2, color=(255,0,0), thickness=4)
    #cv2.drawContours(image, [region['a']], 0, (0,0,255), 2)


for r in reg_pics:
    if isBoard(r):
        findCircles(r['img'], image, r['x'], r['y'])
        cv2.rectangle(image, pt1=r['pt1'], pt2=r['pt2'], color=(0,255,0), thickness=4)
        cv2.imshow('kontury', r['img'])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        cv2.rectangle(image, pt1=r['pt1'], pt2=r['pt2'], color=(0,0,255), thickness=4)

    
cv2.imshow('kontury', image)
cv2.waitKey(0)
cv2.destroyAllWindows()